In [ ]:
import firedrake
from firedrake import Constant
import irksome
from irksome import Dt

In [ ]:
Lx, Ly = Constant(2.0), Constant(1.0)
ny = 64
nx = int(float(Lx / Ly)) * ny
mesh = firedrake.RectangleMesh(
    nx, ny, float(Lx), float(Ly), diagonal='crossed'
)

In [ ]:
import numpy as np
from numpy import pi as π
from firedrake import sqrt, exp, min_value, max_value

def clamp(z, zmin, zmax):
    return min_value(Constant(zmax), max_value(Constant(zmin), z))

def switch(z):
    return exp(z) / (exp(z) + exp(-z))

Ra = Constant(1e6)

ϵ = Constant(1 / nx)
x = firedrake.SpatialCoordinate(mesh)

q = Lx**(7 / 3) / (1 + Lx**4)**(2 / 3) * (Ra / (2 * np.sqrt(π)))**(2/3)
Q = 2 * firedrake.sqrt(Lx / (π * q))
T_u = 0.5 * switch((1 - x[1]) / 2 * sqrt(q / (x[0] + ϵ)))
T_l = 1 - 0.5 * switch(x[1] / 2 * sqrt(q / (Lx - x[0] + ϵ)))
T_r = 0.5 + Q / (2 * np.sqrt(π)) * sqrt(q / (x[1] + 1)) * exp(-x[0]**2 * q / (4 * x[1] + 4))
T_s = 0.5 - Q / (2 * np.sqrt(π)) * sqrt(q / (2 - x[1])) * exp(-(Lx - x[0])**2 * q / (8 - 4 * x[1]))
expr = T_u + T_l + T_r + T_s - Constant(1.5)

In [ ]:
degree = 1
temperature_space = firedrake.FunctionSpace(mesh, 'CG', degree)
T_0 = firedrake.Function(temperature_space)
T_0.interpolate(clamp(expr, 0, 1))
T = T_0.copy(deepcopy=True)

In [ ]:
import matplotlib.pyplot as plt
def subplots():
    fig, axes = plt.subplots()
    axes.set_aspect('equal')
    axes.get_xaxis().set_visible(False)
    axes.get_yaxis().set_visible(False)
    axes.set_xlim(0, float(Lx))
    axes.set_ylim(0, float(Ly))
    return fig, axes

fig, axes = subplots()
firedrake.tripcolor(T, cmap='inferno', axes=axes);

In [ ]:
ρ, c = Constant(1), Constant(1)

In [ ]:
from firedrake import inner
x_0 = Constant((Lx / 2, Ly / 2))
R = Constant(Ly / 4)
r = sqrt(inner(x - x_0, x - x_0))
k_1 = Constant(1 / 16)
k_0 = Constant(1.0)
k = firedrake.conditional(r < R, k_1, k_0)

In [ ]:
from firedrake import inner, grad, dx, ds

ϕ = firedrake.TestFunction(temperature_space)
F = (ρ * c * Dt(T) * ϕ + k * inner(grad(T), grad(ϕ))) * dx

In [ ]:
lower_bc = firedrake.DirichletBC(temperature_space, 1, [3])
upper_bc = firedrake.DirichletBC(temperature_space, 0, [4])
bcs = [lower_bc, upper_bc]

In [ ]:
method = irksome.BackwardEuler()
t = Constant(0.0)
δt = Constant(1e-4)
temperature_solver = irksome.TimeStepper(F, method, t, δt, T, bcs=bcs)

In [ ]:
import tqdm
final_time = 1e-1
num_steps = int(final_time / float(δt))
Ts = [T.copy(deepcopy=True)]
output_freq = 5
for step in tqdm.trange(num_steps):
    temperature_solver.advance()
    if (step + 1) % output_freq == 0:
        Ts.append(T.copy(deepcopy=True))

In [ ]:
%%capture
fig, axes = subplots()
colors = firedrake.tripcolor(
    Ts[0], num_sample_points=4, vmin=0.0, vmax=1.0, cmap='inferno', axes=axes
)

In [ ]:
from matplotlib.animation import FuncAnimation
fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=4)
def animate(T):
    colors.set_array(fn_plotter(T))

animation = FuncAnimation(fig, animate, frames=Ts, interval=1e3 / 24)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())